## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cidreira,BR,-30.1811,-50.2056,71.02,71,0,8.37,clear sky
1,1,Paveh,IR,35.0434,46.3565,38.97,89,50,1.36,scattered clouds
2,2,Ancud,CL,-41.8697,-73.8203,68.52,66,0,2.24,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,60.80,59,0,4.61,clear sky
4,4,Rikitea,PF,-23.1203,-134.9692,77.81,73,96,12.97,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 60


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Ribeira Grande,PT,38.5167,-28.7000,58.66,74,80,13.69,broken clouds
27,27,Ponta Do Sol,PT,32.6667,-17.1000,57.99,74,42,5.99,scattered clouds
54,54,Fortuna,US,40.5982,-124.1573,55.00,81,1,11.50,clear sky
102,102,Abu Kamal,SY,34.4506,40.9171,56.75,58,0,6.71,clear sky
104,104,Praia Da Vitoria,PT,38.7333,-27.0667,57.20,82,40,14.23,scattered clouds
128,128,Vila Franca Do Campo,PT,37.7167,-25.4333,55.99,82,20,6.91,few clouds
177,177,Bomet,KE,-0.7813,35.3416,58.55,94,99,2.84,light rain
183,183,Santander,ES,43.4647,-3.8044,53.60,62,75,12.66,broken clouds
194,194,Ilam,NP,26.9167,87.9167,50.41,54,0,5.39,clear sky
200,200,Khuzdar,PK,27.7384,66.6434,50.95,37,0,3.24,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                68
City                   68
Country                68
Lat                    68
Lng                    68
Max Temp               68
Humidity               68
Cloudiness             68
Wind Speed             68
Current Description    68
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Ribeira Grande,PT,58.66,broken clouds,38.5167,-28.7000,
27,Ponta Do Sol,PT,57.99,scattered clouds,32.6667,-17.1000,
54,Fortuna,US,55.00,clear sky,40.5982,-124.1573,
102,Abu Kamal,SY,56.75,clear sky,34.4506,40.9171,
104,Praia Da Vitoria,PT,57.20,scattered clouds,38.7333,-27.0667,
128,Vila Franca Do Campo,PT,55.99,few clouds,37.7167,-25.4333,
177,Bomet,KE,58.55,light rain,-0.7813,35.3416,
183,Santander,ES,53.60,broken clouds,43.4647,-3.8044,
194,Ilam,NP,50.41,clear sky,26.9167,87.9167,
200,Khuzdar,PK,50.95,clear sky,27.7384,66.6434,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head(10)
clean_hotel_df = hotel_df.dropna()

In [17]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
